In [1]:
import pandas as pd
from math import isnan

In [2]:
GT_plates = pd.read_excel("PlatesLateral.ods", engine="odf")
GT_plates_videos = pd.read_excel("PlatesVideos.ods", engine="odf")
results = pd.read_csv('../out/results_confosable_Bidi.csv')
results_videos = pd.read_csv('../out/results_videos.csv')
#results_times_cpu = pd.read_csv('../out/results_videos.csv')
GT_plates = GT_plates[GT_plates['Plate'].notna()]
GT_plates['Plate'] = GT_plates['Plate'].apply(lambda x: str(x).replace('“','').replace('”',''))
GT_plates

FileNotFoundError: [Errno 2] No such file or directory: '../out/results_confosable_Bidi.csv'

In [ ]:
results

In [ ]:
results_videos

In [ ]:
# Analyze results from images

In [ ]:
n = GT_plates.shape[0]
plates_found = 0
plates_found_by_voting = 0
detected = []
for i,row in GT_plates.iterrows():
    #Get plates recognized in that video
    preds = results.loc[(results['Model'] == row['Model']) & (results['Video'] == row['Video'])]
    #If a frame interval is defined, get only plates in that intervals
    if not isnan(row['Init_frame']):
        preds = preds[preds['Frame'].isin(range(int(row['Init_frame']),int(row['End_frame']+1)))]
    #Check if plate was found
    #print(row['Plate'])
    #print(list(preds['Pred_plate']))
    if row['Plate'] in list(preds['Pred_plate']) or str(row['Plate']).replace('-','') in list(preds['Pred_plate']):
        plates_found+=1
        detected+=['Y']
    else:
        detected+=['N']
    if len(preds['Pred_plate'].mode(dropna=True)):
        democracy_plate = preds['Pred_plate'].mode(dropna=True)[0]
        if democracy_plate == row['Plate']:
            plates_found_by_voting+=1
GT_plates['Detected'] = detected
GT_plates.to_excel("PlatesLateral.ods", engine="odf",index=False)
print(plates_found/n,plates_found_by_voting/n)
GT_plates

In [ ]:
# Analyze results from videos

In [ ]:
n = GT_plates_videos.shape[0]
plates_found = 0
plates_found_by_voting = 0
detected = []
for i,row in GT_plates_videos.iterrows():
    #Get plates recognized in that video
    preds = results_videos.loc[(results_videos['Model'] == row['Model']) & (results_videos['Video'] == row['Video'])]
    #Check if plate was found
    #print(row['Plate'])
    #print(list(preds['Pred_plate']))
    if row['Plate'] in list(preds['Pred_plate']) or str(row['Plate']).replace('-','') in list(preds['Pred_plate']):
        plates_found+=1
        detected+=['Y']
    else:
        detected+=['N']
    if len(preds['Pred_plate'].mode(dropna=True)):
        democracy_plate = preds['Pred_plate'].mode(dropna=True)[0]
        #print(preds['Pred_plate'].mode(dropna=True))
        if democracy_plate == row['Plate']:
            plates_found_by_voting+=1
GT_plates_videos['Detected'] = detected
GT_plates_videos.to_excel("PlatesVideos.ods", engine="odf",index=False)
print(plates_found/n,plates_found_by_voting/n)
GT_plates_videos

In [ ]:
#Analyze times with GPU

In [3]:
results_times_gpu = pd.read_csv('../out/results_times_videos_GPU.csv')
results_times_gpu

FileNotFoundError: [Errno 2] No such file or directory: '../out/results_times_videos_GPU.csv'

In [ ]:
results_times_gpu['Height'].value_counts()

In [ ]:
results_times_gpu['Width'].value_counts()

In [ ]:
results_times_gpu['Detections'].value_counts()

In [ ]:
grouped_df = results_times_gpu.groupby('Detections')

mean_df = grouped_df.mean()

mean_df = mean_df.reset_index()

sd_df = grouped_df.std()

sd_df = sd_df.reset_index()

print(mean_df)
print(sd_df)

In [ ]:
#Analyze times with CPU

In [5]:
results_times_cpu = pd.read_csv('../out/results_times_videos_CPU.csv')
results_times_cpu

,FOTS,Classifier,Refinement,Detections,Width,Height
0,0.432678,0.003759,0.000232,4,1280,704
1,0.375625,0.000677,0.000057,4,1280,704
2,0.381034,0.000689,0.000057,5,1280,704
3,0.430741,0.000724,0.000047,8,1280,704
4,0.421175,0.000707,0.000036,6,1280,704
...,...,...,...,...,...,...
76441,0.409072,0.000415,0.000032,4,1280,704
76442,0.401403,0.000462,0.000031,5,1280,704
76443,0.435094,0.000480,0.000035,6,1280,704
76444,0.462552,0.000533,0.000049,8,1280,704


In [8]:
results_times_cpu['Detections'].value_counts()

2      11091
3      10430
1      10038
4       9366
5       7360
       ...  
174        1
124        1
128        1
101        1
206        1
Name: Detections, Length: 171, dtype: int64

In [6]:
grouped_df = results_times_cpu.groupby('Detections')

mean_df = grouped_df.mean()

mean_df = mean_df.reset_index()

sd_df = grouped_df.std()

sd_df = sd_df.reset_index()

print(mean_df)
print(sd_df)

     Detections      FOTS  Classifier    Refinement        Width      Height
0             0  0.241319    0.000013  2.241701e-07   962.547687  553.647155
1             1  0.338955    0.000429  1.324844e-03  1185.788404  657.871289
2             2  0.377805    0.000506  1.029597e-03  1260.126589  693.454513
3             3  0.391199    0.000549  2.142292e-03  1277.131352  702.361649
4             4  0.398425    0.000591  3.741077e-03  1280.198164  703.733504
..          ...       ...         ...           ...          ...         ...
166         180  1.637590    0.003092  8.409023e-04  1280.000000  704.000000
167         183  1.657038    0.003021  5.695820e-04  1280.000000  704.000000
168         184  1.663996    0.003125  8.119106e-03  1280.000000  704.000000
169         188  1.699984    0.002974  5.531311e-05  1280.000000  704.000000
170         206  1.803712    0.003189  4.212856e-04  1280.000000  704.000000

[171 rows x 6 columns]
     Detections      FOTS  Classifier    Refinement 